In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}

In [2]:
import Text.Parsec 
import Text.ParserCombinators.Parsec.Number
import qualified Data.Map.Strict as M

In [3]:
data Instruction = Instruction 
                    { register :: String
                    , direction :: String
                    , change :: Int
                    , conditionRegister :: String
                    , operation :: String
                    , comparator :: Int
                    } deriving (Show, Eq)

In [4]:
type Memory = M.Map String Int

In [5]:
onlySpaces = many (oneOf " \t")
symP = (many lower) <* onlySpaces
operationP = (many1 (oneOf "!<>=")) <* onlySpaces

In [6]:
iFile = iLine `sepBy` newline 
iLine = instructify <$> symP 
                    <*> symP 
                    <*> int 
                    <*> ( onlySpaces *> string "if" *> onlySpaces *> symP )
                    <*> operationP 
                    <*> int
    where instructify r d c cr o p = Instruction { register = r
                                                 , direction = d
                                                 , change = c
                                                 , conditionRegister = cr
                                                 , operation = o
                                                 , comparator = p
                                                 }

In [7]:
parseFile :: String -> Either ParseError [Instruction]
parseFile input = parse iFile "(unknown)" input

parseLine :: String -> Either ParseError Instruction
parseLine input = parse iLine "(unknown)" input

successfulParse :: Either ParseError [a] -> [a]
successfulParse (Left _) = []
successfulParse (Right a) = a

In [8]:
parseLine "b inc 5 if a > 1"

Right (Instruction {register = "b", direction = "inc", change = 5, conditionRegister = "a", operation = ">", comparator = 1})

In [9]:
sampleT = "b inc 5 if a > 1\na inc 1 if b < 5\nc dec -10 if a >= 1\nc inc -20 if c == 10"

In [10]:
sample = successfulParse $ parseFile sampleT
sample

[Instruction {register = "b", direction = "inc", change = 5, conditionRegister = "a", operation = ">", comparator = 1},Instruction {register = "a", direction = "inc", change = 1, conditionRegister = "b", operation = "<", comparator = 5},Instruction {register = "c", direction = "dec", change = -10, conditionRegister = "a", operation = ">=", comparator = 1},Instruction {register = "c", direction = "inc", change = -20, conditionRegister = "c", operation = "==", comparator = 10}]

In [11]:
conditionEval :: Int -> String -> Int -> Bool
conditionEval reg op val
    | op == "==" = reg == val
    | op == "<"  = reg < val
    | op == ">"  = reg > val
    | op == "<=" = reg <= val
    | op == ">=" = reg >= val
    | op == "!=" = reg /= val

In [12]:
-- effectiveChange :: String -> Int -> Int
-- effectiveChange dir val
--     | dir == "inc" = val
--     | dir == "dec" = - val

In [30]:
effectiveChange :: String -> Int -> Int
effectiveChange "inc" val = val
effectiveChange "dec" val = -val

In [32]:
processInstruction memory instruction = memory'
    where v = M.findWithDefault 0 (register instruction) memory
          cv = M.findWithDefault 0 (conditionRegister instruction) memory
          condition = conditionEval cv (operation instruction) (comparator instruction)
          delta = effectiveChange (direction instruction) (change instruction)
          memory' = if condition
                    then M.insert (register instruction) (v + delta) memory
                    else memory

In [33]:
processInstructions = foldl processInstruction M.empty 

In [34]:
processInstructions sample

fromList [("a",1),("c",-10)]

In [47]:
largestValue m 
    | M.null m = 0
    | otherwise = maximum $ M.elems m

In [48]:
largestValue $ processInstructions sample

1

In [49]:
part1 = largestValue . processInstructions

In [50]:
-- part1 = processInstructions

In [51]:
main :: IO ()
main = do 
        text <- readFile "../../data/advent08.txt"
        let instrs = successfulParse $ parseFile text
        print $ part1 instrs

In [52]:
main

4647

In [53]:
processInstructionH (highest, memory) instruction = (highest', memory')
    where memory' = processInstruction memory instruction
          h = largestValue memory'
          highest' = if h > highest then h else highest

In [54]:
processInstructionsH = foldl processInstructionH (0, M.empty)

In [55]:
processInstructionsH sample

(10,fromList [("a",1),("c",-10)])

In [56]:
part2 = fst . processInstructionsH

In [57]:
main :: IO ()
main = do 
        text <- readFile "../../data/advent08.txt"
        let instrs = successfulParse $ parseFile text
        print $ part1 instrs
        print $ part2 instrs

In [58]:
main

4647
5590